<a href="https://colab.research.google.com/github/cosimo-schiavoni/Massive_Data_Project/blob/main/Downloads_HOROVOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
!wget -q https://www-eu.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!rm spark-3.2.1-bin-hadoop2.7.tgz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install findspark
!pip install pyspark
!pip install -q kaggle
!pip install elephas
!pip install sparkdl
!pip install tensorframes
!pip install petastorm

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 45.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f38d2ce4f45a9f52d1a2ababc6629124bc1f41b928d5f7276ad26868ebcfc59d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 4.1 MB 7.5 MB/s 
     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 35.4 MB/s 
     |████████████████████████████████| 462 kB 61.2 MB/s 
  Created wheel for elephas: filename=elephas-3.1.0-py3-none-any.whl size=26259 sha256=7be1b36a18c139d956a2f982b1d4a1984e8a32b641e29e97ebf65264dae62cab
  Stored in directory: /root/.cache/pip/wheels/33/e4/e1/56dda8be927bb0e9971cd7ddf3fc1b17ce78db56268b1f867f
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=130fc

In [1]:
import pyspark
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.mllib.evaluation import MulticlassMetrics

# Keras / Deep Learning
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers, regularizers
from tensorflow.keras.optimizers import Adam

# Elephas for Deep Learning on Spark
from elephas.ml_model import ElephasEstimator

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from functools import reduce
from google.colab import files
import os
import zipfile
import findspark
from pyspark.sql.functions import col
from petastorm import TransformSpec
from petastorm.spark import SparkDatasetConverter, make_spark_converter
import numpy as np

#from my model
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import pickle
import shutil
import random
import skimage.io as io
from copy import deepcopy



/usr/local/lib/python3.7/dist-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


In [4]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

Saving kaggle.json to kaggle.json


In [5]:
os.environ["data_source"] = "./Face_Comics_data"

api = KaggleApi()
api.authenticate()

if not os.path.exists(os.environ["data_source"]):
  os.makedirs(os.environ["data_source"])
  if "comic-faces-paired-synthetic-v2" not in os.listdir():
    if "comic-faces-paired-synthetic-v2.zip" not in os.listdir():
      ! kaggle datasets download -d defileroff/comic-faces-paired-synthetic-v2
      with zipfile.ZipFile("comic-faces-paired-synthetic-v2.zip", 'r') as f:
        f.extractall("comic-faces-paired-synthetic-v2")
    os.remove("comic-faces-paired-synthetic-v2.zip")


!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/comics" "./Face_Comics_data"
!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/faces" "./Face_Comics_data"

!rm -rf comic-faces-paired-synthetic-v2

100% 2.17G/2.18G [00:15<00:00, 82.3MB/s]
100% 2.18G/2.18G [00:15<00:00, 151MB/s] 


In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

findspark.init(findspark.find())
#findspark.init(findspark.find())
#spark = SparkSession.builder.master("local").appName("Colab").getOrCreate()
#spark

spark = SparkSession.builder.master("local[*]").appName("Colab").getOrCreate()
conf = SparkConf().setMaster("local[*]").setAppName("Colab")
conf.set("spark.executor.memory", "6G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "4")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.default.parallelism", "4")
sc =  SparkContext.getOrCreate(conf=conf)
spark

In [4]:
sql_context = SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)

In [5]:
df_comics = spark.read.format("image").load(".//Face_Comics_data//comics").withColumn("label", lit(int(1)))
df_faces = spark.read.format("image").load(".//Face_Comics_data//faces").withColumn("label", lit(int(0)))


df_comics = df_comics.select(col("image.data"),col("label"))
df_faces = df_faces.select(col("image.data"),col("label"))

#df_comics.show(5)
#df_faces.show(5)

In [6]:
#Read images and Create training & test DataFrames for transfer learning
#Shuffle Data
df_comics = df_comics.orderBy(rand())
df_faces = df_faces.orderBy(rand())

comics_train, comics_validation, comics_test = df_comics.randomSplit([0.6, 0.2,0.2], seed = 1234)
print(comics_train)
faces_train, faces_validation, faces_test = df_faces.randomSplit([0.6, 0.2,0.2], seed = 1234)
print(faces_train)

#dataframe for training a classification model
train_df = faces_train.unionAll(comics_train)

#dataframe for validating the classification model
val_df = faces_validation.unionAll(comics_validation)

#dataframe for testing the classification model
test_df = faces_test.unionAll(comics_test)

num_classes = 2
train_df = train_df.repartition(2)

DataFrame[data: binary, label: int]
DataFrame[data: binary, label: int]


In [ ]:
! rm -rf './staging'

In [7]:
from PIL import Image
import io

def preprocess(content):
  image = Image.open(io.BytesIO(content)).resize([224,224])
  image_array = keras.preprocessing.image.img_to_array(image)
  #return tf.keras.applications.mobilnte_v2.preprocess_input(image_array)
  return image_array

def transform_batch(pd_batch):
  pd_batch['features'] = pd_batch['content'].map(lambda x: preprocess(x))
  pd_batch = pd_batch.drop(labels='content', axis = 1)
  return pd_batch

IMG_SHAPE = (224, 224, 3)
EPOCHS = 15
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

transform_spec_fn = TransformSpec(
    transform_batch,
    edit_fields = [('features', np.float32, IMG_SHAPE, False)],
    selected_fields = ['features','label']
)

In [ ]:
#import pandas as pd
#pd_batch0 = train_df.toPandas()
#pd_batch0

,content,label
0,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0
1,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0
2,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0
3,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",1
4,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",1
5,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",1
6,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0
7,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0
8,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",0
9,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...",1


In [9]:
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, "file:/content/staging")

converter_train = make_spark_converter(train_df)
converter_val = make_spark_converter(val_df)

KeyboardInterrupt: ignored

In [ ]:
#data augmentation


def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (180-x)
  else:
    x
  return x

###Randim invert
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))


class RandomInvert(layers.Layer):
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor

  def call(self, x):
    return random_invert_img(x)


###Kernel Inizializer Sobel_x
def hedge_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [-5, -4, 0, 4, 5], 
            [-8, -10, 0, 10, 8], 
            [-10, -20, 0, 20, 10], 
            [-8, -10, 0, 10, 8], 
            [-5, -4, 0, 4, 5]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (5, 5, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x


def vertical_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [1, 0, -1], 
            [1, 0, -1], 
            [1, 0, -1]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

def horizontal_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [1, 1, 1], 
            [0, 0, 0], 
            [-1, -1, -1]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

In [8]:
def get_model():
  cnn = tf.keras.models.Sequential()
  cnn.add(tf.keras.layers.Conv2D(32, kernel_size=5, strides=2, activation='relu',  input_shape = [224,224,3]))
  cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
  cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,  strides=2, activation = 'relu'))
  cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
  cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2,  strides=2, activation = 'relu'))
  cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=1, padding='same'))
  cnn.add(tf.keras.layers.Flatten())
  cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
  cnn.add(tf.keras.layers.Dropout(0.1))
  cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
  return cnn

def get_compiled_model():
  cnn = get_model()
  cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return cnn

In [ ]:
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, "file:/content/staging")

converter_train = make_spark_converter(train_df)
converter_val = make_spark_converter(val_df)

/usr/local/lib/python3.7/dist-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
Converting floating-point columns to float32
Converting floating-point columns to float32
The median size 5737694 B (< 50 MB) of the parquet files is too small. Total size: 464958137 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:///content/staging/20220514142256-appid-local-1652536847498-6d5d3be3-0aef-44af-bfb5-2949cf719483/part-00067-08e6490c-83ac-4bd6-b63f-494523a9d1ab-c000.parquet, ...


In [21]:
!pip install horovod[tensorflow,keras,pytorch,mxnet,spark]

     |████████████████████████████████| 47.3 MB 74 kB/s 
     |████████████████████████████████| 813 kB 40.3 MB/s 
     |████████████████████████████████| 409 kB 54.9 MB/s 
     |████████████████████████████████| 829 kB 40.9 MB/s 
     |████████████████████████████████| 636 kB 42.2 MB/s 
     |████████████████████████████████| 1.1 MB 50.4 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 144 kB 54.5 MB/s 
     |████████████████████████████████| 271 kB 55.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=88fffb4c05780cb816586770f8d697cc7d535f0e05ed4513a668af3d5862f975
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: graphviz
    Found exis

In [13]:
!pip install tensorflowonspark

     |████████████████████████████████| 45 kB 1.7 MB/s 


In [9]:
import tensorflowonspark

In [15]:
import sys
try:
  from sparkdl import HorovodRunner
  raise
except Exception as err:
      exc_type, exc_obj, exc_tb = sys.exc_info()

str(exc_obj)

"cannot import name 'HorovodRunner' from 'sparkdl' (/usr/local/lib/python3.7/dist-packages/sparkdl/__init__.py)"

In [73]:
with open("/usr/local/lib/python3.7/dist-packages/sparkdl/transformers/keras_applications.py", "r") as file:
  data = file.read()
# Replace matches
data = data.replace("keras.applications", "tensorflow.keras.applications")
# Write the data back to the file
with open("/usr/local/lib/python3.7/dist-packages/sparkdl/transformers/keras_applications.py", "w") as file:
  file.write(data)

In [69]:
!ls "/usr/local/lib/python3.7/dist-packages/sparkdl/transformers/"

easy_feature.py  keras_applications.py	keras_utils.py	__pycache__  tf_text.py
__init__.py	 keras_image.py		named_image.py	tf_image.py  utils.py


In [74]:
with open("/usr/local/lib/python3.7/dist-packages/sparkdl/transformers/keras_applications.py", "r") as file:
  data = file.read()
data

'# Copyright 2017 Databricks, Inc.\n#\n# Licensed under the Apache License, Version 2.0 (the "License");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n#   http://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an "AS IS" BASIS,\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n# See the License for the specific language governing permissions and\n# limitations under the License.\n#\n\nfrom abc import ABCMeta, abstractmethod\n\nimport keras.backend as K\nfrom tensorflow.keras.applications import inception_v3, xception, resnet50\nimport numpy as np\nimport tensorflow as tf\n\nfrom sparkdl.transformers.utils import (imageInputPlaceholder, InceptionV3Constants)\n\n\n"""\nEssentially a factory function for getting the correct KerasApplicationModel class\nfor the network name.\n"""\ndef

In [79]:
#!pip install kafka
!pip install kafka-python

     |████████████████████████████████| 246 kB 5.2 MB/s 


In [81]:
from kafka import KafkaConsumer

ImportError: ignored

In [10]:
from sparkdl import HorovodRunner

ImportError: ignored

In [11]:
import horovod.tensorflow.keras as hvd
#from sparkdl import HorovodRunner

In [12]:
def training_and_validation():

  hvd.int()
  
  #pin GPU
  gpus = tf.config.experimental.list_physical_devices('GPU')
  for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
  if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

  model = get_model(lr)

  #Adjust learning rate basing on GPU
  optimizer = keras.optimizers.SGD(lr = lr * hvd.size(), momentum = 0.9)
  dist_optimizer = hdv.DistributedOptimizer(optimizer)

  callbacks = [
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),
    hvd.callbacks.MetricAverageCallback(),
  ]


  cnn = get_compiled_model()
  
  with  converter_train.make_tf_dataset(transform_spec=transform_spec_fn, 
                                    cur_shard = hvd.rank(), shard_count = hvd.size(),
                                    batch_size = BATCH_SIZE) as train_dataset,\
        converter_val.make_tf_dataset(transform_spec=transform_spec_fn, 
                                    batch_size = BATCH_SIZE) as val_dataset:
                                    
      train_dataset = train_dataset.map(lambda x: (x.features, x.label))
      val_dataset = val_dataset.map(lambda x: (x.features, x.label))
      
      hist = cnn.fit(train_dataset, validation_data = val_dataset, epochs = EPOCHS)
      return hist.history['val_loss'][-1], hist.history['val_accuracy'][-1]

loss, accuracy = training_and_validation()
print('Validation accuracy: {}'.format(accuracy))


AttributeError: ignored

In [ ]:
hr = horovodRunner(np = 2)
hr.run(training_and_validation)